This file trying to extract PSD and DE from EEG data.

## CONFIGURATIONS

In [4]:
CONFIG = {
    "waching_data_path": "data\sliced_eeg\watching", 
    "imaging_data_path": "data\sliced_eeg\imaging",
    "watching_PSD_DE": "data\PSD_DE\watching",
    "imaging_PSD_DE": "data\PSD_DE\imaging", 
    "freq_band": [(1, 4), (4, 8), (8, 14), (14, 31), (31, 99)], 
    "watching_time": 2, 
    "imaging_time": 3, 
    "frequency": 200
}

'''
1-4Hz corresponds to delta band
4-8Hz corresponds to theta band
8-14Hz corresponds to alpha band
14-31Hz corresponds to beta band
31-99Hz corresponds to gamma band
'''

'\n1-4Hz corresponds to delta band\n4-8Hz corresponds to theta band\n8-14Hz corresponds to alpha band\n14-31Hz corresponds to beta band\n31-99Hz corresponds to gamma band\n'

## Extract PSD and DE from eeg data

In [5]:
import numpy as np

def DE_PSD(data, time_win, fre=CONFIG["frequency"], STFTN = 200, freq_band = CONFIG["freq_band"]): 
    '''
    compute the differential entropy(DE) and power spectral density(PSD) of EEG signal
    
    args:
    data: EEG signal, (2*200) or (3*200)
    fre: sampling frequency
    time_win: time window for calculating PSD
    STFTN: number of points for Short-Time Fourier, which will influence the frequency resolution
    freq_band: frequency bands for calculating PSD

    
    returns:
    DE: differential entropy, (num_electrodes=62, len(freq_band)=5)
    PSD: power spectral density, (num_electrodes=62, len(freq_band)=5)
    '''

    # turn freq_band into np array
    fStart, fEnd = list(zip(*freq_band))
    fStart, fEnd = np.array(fStart), np.array(fEnd)
    fStartNum = (fStart/fre*STFTN).astype(int)
    fEndNum = (fEnd/fre*STFTN).astype(int)

    # calculate Hann window for smoothing
    Hlength = int(time_win*fre)
    Hwindow = np.hanning(Hlength)
    
    
    num_electrodes = data.shape[0]
    psd = np.zeros((num_electrodes, len(freq_band)))
    de = np.zeros((num_electrodes, len(freq_band)))

    for i in range(num_electrodes):
        data_i = data[i,:]
        Hdata = data_i*Hwindow
        FFTdata = np.fft.fft(Hdata, n=STFTN)
        # input of fft is real signal, so only need to calculate half of the spectrum
        magFFTdata = abs(FFTdata[0:int(STFTN/2)])

        # calculate PSD DE
        for p in range(0,len(freq_band)):
            E = 0
            for p0 in range(fStartNum[p],fEndNum[p]+1):
                E=E+magFFTdata[p0]*magFFTdata[p0]
            E = E/(fEndNum[p]-fStartNum[p]+1)
            psd[i][p] = E
            de[i][p] = np.log2(100*E)
        
    return de, psd

def extract_features(clip_segments, time_win):
    DE_list = []
    PSD_list = []
    for i in range(0,clip_segments.shape[0]):
        DE_list_i = []
        PSD_list_i = []
        for j in range(0, clip_segments.shape[1]):
            DE, PSD = DE_PSD(clip_segments[i,j,:], time_win)
            DE_list_i.append(DE)
            PSD_list_i.append(PSD)
        DE_list.append(np.stack(DE_list_i, 0))
        PSD_list.append(np.stack(PSD_list_i, 0))
    DE_list = np.stack(DE_list, 0)
    PSD_list = np.stack(PSD_list, 0)
    PSD_DE_data = np.stack((PSD_list, DE_list))
    return PSD_DE_data

## Start!

In [ ]:
import os
from tqdm import tqdm

for file in tqdm(os.listdir(CONFIG["waching_data_path"])):
    if file.endswith(".npy"):
        watching_data = np.load(os.path.join(CONFIG["waching_data_path"], file))
        print(watching_data.shape)
        watching_PSD_DE = extract_features(watching_data, time_win=CONFIG["watching_time"])
        np.save(os.path.join(CONFIG["watching_PSD_DE"], file.replace(".npy", "_PSD_DE.npy")), watching_PSD_DE)

for file in tqdm(os.listdir(CONFIG["imaging_data_path"])):
    if file.endswith(".npy"):
        imaging_data = np.load(os.path.join(CONFIG["imaging_data_path"], file))
        imaging_PSD_DE = extract_features(imaging_data, time_win=CONFIG["imaging_time"])
        np.save(os.path.join(CONFIG["imaging_PSD_DE"], file.replace(".npy", "_PSD_DE.npy")), imaging_PSD_DE)

print("="*50, "\nAll files are done!")

  0%|          | 0/6 [00:00<?, ?it/s]

(5, 50, 62, 400)


 17%|█▋        | 1/6 [00:00<00:02,  1.95it/s]

(5, 50, 62, 400)


 33%|███▎      | 2/6 [00:01<00:02,  1.88it/s]

...

 67%|██████▋   | 4/6 [00:02<00:01,  1.90it/s]

(5, 50, 62, 400)


 83%|████████▎ | 5/6 [00:02<00:00,  1.89it/s]

(5, 50, 62, 400)


100%|██████████| 6/6 [00:03<00:00,  1.84it/s]
